In [ ]:
import os
import pandas as pd
from pathlib import Path
from IPython.display import display, HTML

# Directorio base donde están las carpetas
base_dir = Path('../data')

# Subcarpetas que queremos analizar y sus subdirectorios
folder_mapping = {
    'ree/data_parquet_clean/almacenamiento': 'almacenamiento',
    'ree/data_parquet_clean/demanda': 'demanda',
    'ree/data_parquet_clean/emision': 'emision',
    'ree/data_parquet_clean/generacion': 'generacion',
    'hidrografica/data_parquet_clean/embalses': 'embalses',
    'climatologia/data_parquet_clean/meteo': 'meteo',
    'precio_luz/data_parquet_clean/precios_luz': 'precios_luz'
}

# Función para obtener el resumen del dataset
def get_dataset_summary(file_path):
    df = pd.read_parquet(file_path)
    
    # Encontrar la columna de tiempo
    time_cols = [col for col in df.columns if any(term in col.lower() 
                for term in ['hora', 'time', 'fecha', 'date'])]
    
    # Obtener el nombre de la tabla basado en la ruta
    rel_path = str(file_path.relative_to(base_dir))
    tabla_nombre = None
    for path_pattern, name in folder_mapping.items():
        if path_pattern in rel_path:
            tabla_nombre = name
            break
    
    if tabla_nombre is None:
        return None
    
    # Obtener el año de partición si existe
    try:
        años = df[time_cols[0]].dt.year.unique() if time_cols else []
        años_str = ', '.join(map(str, sorted(años)))
    except:
        años_str = 'No disponible'
    
    summary = {
        'Tabla': tabla_nombre,
        'Años': años_str,
        'Num. Registros': len(df)
    }
    
    if time_cols:
        time_col = time_cols[0]
        summary['Columna Temporal'] = time_col
        summary['Fecha Mínima'] = df[time_col].min()
        summary['Fecha Máxima'] = df[time_col].max()
    
    return pd.DataFrame([summary])

# Función para obtener los primeros y últimos registros
def get_dataset_samples(file_path):
    df = pd.read_parquet(file_path)
    
    # Obtener el nombre de la tabla basado en la ruta
    rel_path = str(file_path.relative_to(base_dir))
    tabla_nombre = None
    for path_pattern, name in folder_mapping.items():
        if path_pattern in rel_path:
            tabla_nombre = name
            break
    
    if tabla_nombre is None:
        return None
    
    # Combinar los primeros y últimos registros
    sample_df = pd.concat([df.head(2), df.tail(2)])
    sample_df.insert(0, 'Tabla', tabla_nombre)
    
    # Encontrar y añadir el año si hay columna temporal
    time_cols = [col for col in df.columns if any(term in col.lower() 
                for term in ['hora', 'time', 'fecha', 'date'])]
    if time_cols:
        time_col = time_cols[0]
        sample_df.insert(1, 'Año', sample_df[time_col].dt.year)
    
    return sample_df

# Listas para almacenar resultados
summaries = []
samples = []

# Buscar todos los archivos parquet en el directorio base
all_parquet_files = list(base_dir.rglob('*.parquet'))

# Procesar cada archivo
for parquet_file in all_parquet_files:
    try:
        summary_df = get_dataset_summary(parquet_file)
        if summary_df is not None:
            summaries.append(summary_df)
            
        sample_df = get_dataset_samples(parquet_file)
        if sample_df is not None:
            samples.append(sample_df)
            
    except Exception as e:
        print(f"Error al procesar {parquet_file}: {str(e)}")

# Combinar todos los resultados
all_summaries = pd.concat(summaries, ignore_index=True) if summaries else pd.DataFrame()
all_samples = pd.concat(samples, ignore_index=True) if samples else pd.DataFrame()

# Ordenar los resultados usando un orden específico para las tablas
orden_tablas = ['almacenamiento', 'demanda', 'emision', 'generacion', 'embalses', 'meteo', 'precios_luz']
all_summaries['Tabla'] = pd.Categorical(all_summaries['Tabla'], categories=orden_tablas, ordered=True)
all_samples['Tabla'] = pd.Categorical(all_samples['Tabla'], categories=orden_tablas, ordered=True)

# Ordenar los resultados
all_summaries = all_summaries.sort_values(['Tabla'])
all_samples = all_samples.sort_values(['Tabla', 'Año'])

# Resumen de los Datasets

A continuación se muestra un resumen de todos los datasets encontrados, incluyendo sus fechas mínimas y máximas, y el número total de registros.

In [2]:
# Mostrar resumen de datasets
display(HTML(all_summaries.to_html(index=False, 
                                 float_format=lambda x: f"{x:,.0f}" if isinstance(x, (int, float)) else x)))

Tabla,Años,Num. Registros,Columna Temporal,Fecha Mínima,Fecha Máxima
almacenamiento,2025,84672,Hora,2025-01-01,2025-10-21 23:55:00
almacenamiento,2023,105120,Hora,2023-01-01,2023-12-31 23:55:00
almacenamiento,2024,105408,Hora,2024-01-01,2024-12-31 23:55:00
almacenamiento,2021,105120,Hora,2021-01-01,2021-12-31 23:55:00
almacenamiento,2020,105408,Hora,2020-01-01,2020-12-31 23:55:00
almacenamiento,2022,105120,Hora,2022-01-01,2022-12-31 23:55:00
demanda,2022,105120,Hora,2022-01-01,2022-12-31 23:55:00
demanda,2023,105120,Hora,2023-01-01,2023-12-31 23:55:00
demanda,2024,105408,Hora,2024-01-01,2024-12-31 23:55:00
demanda,2021,105120,Hora,2021-01-01,2021-12-31 23:55:00


In [3]:
# Mostrar all_sumaries agrupado por Tabla y Años, sumando registros, maximo de columna temporal, minimo de Fecha Minima y maximo de Fecha Máxima
grouped_summary = all_summaries.groupby(['Tabla']).agg({
    'Num. Registros': 'sum',
    'Fecha Mínima': 'min',
    'Fecha Máxima': 'max'
}).reset_index()

grouped_summary

/var/folders/z9/456rf8tn1kv_0wm71fpsqcyc0000gn/T/ipykernel_2124/1990634747.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_summary = all_summaries.groupby(['Tabla']).agg({


,Tabla,Num. Registros,Fecha Mínima,Fecha Máxima
0,almacenamiento,610848,2020-01-01,2025-10-21 23:55:00
1,demanda,610848,2020-01-01,2025-10-21 23:55:00
2,emision,610848,2020-01-01,2025-10-21 23:55:00
3,generacion,610848,2020-01-01,2025-10-21 23:55:00
4,embalses,1030824,2020-01-09,2025-10-06 00:00:00
5,meteo,2338272,2020-01-01,2025-10-18 23:00:00
